<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/3_Embeddings/ejercicios/ejercicios_solucion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios clase 3

En este notebook vamos a utilizar embeddings GloVe preentrenados para mejorar la performance de nuestro modelo de análisis de sentimiento de la clase 2. 

Comenzaremos por descargar los embeddings. Los autores del paper de GloVe han lanzado cuatro archivos de texto con vectores de palabras entrenados en diferentes conjuntos de datos web masivos. Están disponibles para descargar [aquí](https://nlp.stanford.edu/projects/glove/).

Usaremos “Wikipedia 2014 + Gigaword 5” que es el archivo más pequeño (“glove.6B.zip”) con 822 MB. Fue entrenado en un corpus de 6 mil millones de tokens y contiene un vocabulario de 400 mil tokens.

La siguiente celda descarga y descomprime el archivo.

In [64]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-08-24 01:08:32--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-24 01:08:33--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  5.07MB/s    in 2m 39s  

2022-08-24 01:11:13 (5.17 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: glo

Después de descomprimir el archivo descargado, encontramos cuatro archivos txt: glove.6B.50d.txt, glove.6B.100d.txt, glove.6B.200d.txt, glove.6B.300d.txt. Como sugiere su nombre de archivo, tienen vectores con diferentes dimensiones.

Si imprimimos las primeras lineas podemos notar que cada linea contiene un embedding en donde el primer elemento es un tensor y los siguientes son las componentes del vector.

In [65]:
with open( "glove.6B.100d.txt", 'r') as f:
  n_lin = 0
  for line in f:
    print(line)
    n_lin += 1
    if n_lin>3: break

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

, -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -0.52158

# Ejercicio 1 

Cargue los primeros 400.000 embeddings del archivo de 100 dimensiones en memoria tal como lo hicimos en la clase 3, parte 3.

Nota: tener en cuenta que este archivo no tiene una primera linea de encabezados como el archivo de la clase.*texto en cursiva*

In [66]:
# inserte su código aquí

import torch
class TokenEmbedding:
  """Token Embedding."""
  def __init__(self, file_name, n):
    self.idx_to_token, self.idx_to_vec, self.dim = self._load_embedding(
        file_name, n)
    self.unknown_idx = 0
    self.token_to_idx = {token: idx for idx, token in
                          enumerate(self.idx_to_token)}
  

  def _load_embedding(self, file_name, n):
    idx_to_token, idx_to_vec = ['<unk>'], []
    with open( file_name, 'r') as f:
      i=0
      for line in f:
        if n<i: break
        else: i+=1
        elems = line.rstrip().split(' ')
        token, elems = elems[0], [float(elem) for elem in elems[1:]]
        # Skip header information, such as the top row in fastText
        if len(elems) > 1:
            idx_to_token.append(token)
            idx_to_vec.append(elems)
    idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
    return idx_to_token, torch.tensor(idx_to_vec), len(idx_to_vec[0])

  def __getitem__(self, tokens):
    indices = [self.token_to_idx.get(token, self.unknown_idx)
                for token in tokens]
    vecs = self.idx_to_vec[torch.tensor(indices)]
    return vecs

  def __len__(self):
    return len(self.idx_to_token)

english_glove_100d = TokenEmbedding("glove.6B.100d.txt",400000)

# Ejercicio 2

Modificar el modelo `model2`, definido en los ejercicios de la clase 2 para análisis de sentimientos, para que la capa de embedding se inicialice con los embeddings GloVe preentrenados que descargamos.

Nota: Recuerde que la matriz de embeddings depende del vocabulario a usar en el problema. Por lo tanto, deberá realizar todo el preprocesamiento de los datos, para obtener el vocabulario, antes de modificar el modelo. 

In [67]:
# inserte su código a partir de aquí

In [68]:
################################ Setup
import torch
import torchtext

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
################################ Preprocesamos los datos
import torchdata
import random
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

train_data, test_data = IMDB(split=('train', 'test'))

# guardaremos momentaneamente los datos en listas.
full_list = list(train_data)
cutoff = int(len(full_list) * 0.7)

random.seed(SEED)
random.shuffle(full_list)

train_list = full_list[:cutoff] # entrenamiento
val_list = full_list[cutoff:] # validacion
test_list = list(test_data) # prueba

#Generamos el vocabulario
tokenizer = get_tokenizer('spacy', 'en_core_web_sm')
counter = Counter()
for (label, line) in train_list: 
    counter.update(tokenizer(line)) #contador como el visto en clase
vocab = vocab(counter, min_freq=10, # eliminamos todo menor a 10 repeciones
              specials=('<unk>', '<PAD>'))
vocab.set_default_index(vocab['<unk>'])


In [70]:
################################ Modificamos el modelo
import numpy as np
import torch.nn as nn

#Generamos la matriz de pesos a partir del vocabulario con los embeddings GloVe
matrix_len = len(vocab.get_itos())
weights_matrix = np.zeros((len(vocab.get_itos()), english_glove_100d.dim))
words_found = 0
words_not_found = []

for i, word in enumerate(vocab.get_itos()):
    try: 
        weights_matrix[i] = english_glove_100d[[word]][0]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(english_glove_100d.dim, ))
        words_not_found.append(word)

#Definimos la función que crea la capa Embedding con la matriz de pesos
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': torch.tensor(weights_matrix)})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

#Modificamos el modelo para inicializar la capa Embedding
class RNN(nn.Module):
    def __init__(self, input_dim, weights_matrix, pad_index,
                 hidden_dim, output_dim, num_layers, bidirectional, dropout):
        
        super().__init__()

        self.dropout = nn.Dropout(dropout)

        ###### inicio modificación #######
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix,
                                                                         True)
        ###### fin modificación #######
            
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers,
                          bidirectional = bidirectional, dropout=dropout)

        self.num_layers = num_layers
        self.bidirectional = bidirectional        
        if bidirectional:
          hidden_dim *= 2
        self.linear = nn.Linear(hidden_dim * num_layers, output_dim)

        
    def forward(self, text, text_lengths):

        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        #embedded = [sent len, batch size, emb dim]
        
        packed_output, hidden = self.rnn(packed_embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [2 * 2, batch size, hid dim]
        chunks = self.num_layers
        if self.bidirectional: chunks *= 2
        dropped_h = self.dropout(
            torch.cat(torch.chunk(hidden,chunks,0), dim=2).squeeze(0))
        return self.linear(dropped_h)


# Instanciamos el modelo definido
INPUT_DIM = len(vocab)
PAD_IDX = vocab['<PAD>']
HIDDEN_DIM = 256
NUM_LAYERS = 2
BIDIRECTIONAL = True
OUTPUT_DIM = 1
DROPOUT = 0.5

model2 = RNN(INPUT_DIM, weights_matrix, PAD_IDX,
            HIDDEN_DIM,OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model2):,} parámetros entrenables')


El modelo tiene 1,733,633 parámetros entrenables


# Ejercicio 3 
Entrene este nuevo modelo y compare los resultados obtenidos.

Nota: Reutilice todo el código de los ejercicios de la clase 2

In [71]:
# inserte su código a partir de aquí

In [72]:
####################### Creación de Dataloaders


import random
from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

class BucketSampler(Sampler):

    def __init__(self, batch_size, train_list):
        self.length = len(train_list)
        self.train_list = train_list
        self.batch_size = batch_size
        indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(self.train_list)]
        random.seed(SEED)
        random.shuffle(indices)
        pooled_indices = []
        # creamos minilotes de tamaños similares
        for i in range(0, len(indices), self.batch_size * 100):
            pooled_indices.extend(sorted(indices[i:i + self.batch_size * 100], key=lambda x: x[1], reverse=True))

        self.pooled_indices = pooled_indices

    def __iter__(self):
        for i in range(0, len(self.pooled_indices), self.batch_size):
            yield [idx for idx, _ in self.pooled_indices[i:i + self.batch_size]]

    def __len__(self):
        return self.length
        return (self.length + self.batch_size - 1) // self.batch_size

text_transform = lambda x: [vocab[token] for token in tokenizer(x)]
label_transform = lambda x: 1 if x == 'pos' else 0

def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        # convertimos la etiqueta usando label_transform
        label_list.append(label_transform(_label))
        # convertimos el texto en tokens
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
        # guardamos la longitud de cada token
        length_list.append(len(processed_text))
    # armamos la tupla que conformara un ejemplo de minilote.
    result = (torch.tensor(label_list),
              pad_sequence(text_list, padding_value=1.0),
              torch.tensor(length_list) )
    return result

batch_size = 64  # A batch size of 64

train_bucket = BucketSampler(batch_size, train_list)
train_iter = DataLoader(train_list,
                          batch_sampler=train_bucket,
                          collate_fn=collate_batch)

val_bucket = BucketSampler(batch_size, val_list)
val_iter = DataLoader(val_list,
                          batch_sampler=val_bucket,
                          collate_fn=collate_batch)

test_bucket = BucketSampler(batch_size, test_list)
test_iter = DataLoader(test_list,
                          batch_sampler=test_bucket,
                          collate_fn=collate_batch)


In [73]:
###################### Preparación del entrenamiento
import torch.optim as optim
import time

optimizer2 = optim.Adam(model2.parameters())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model2 = model2.to(device)
loss = nn.BCEWithLogitsLoss().to(device)

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convertimos a flotante para la división
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train_epoch_wl(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    stop = 0
    for label, text, length in iterator:
        label = label.float().to(device)
        text = text.to(device)
        optimizer.zero_grad()       
        predictions = model(text, length).squeeze(1)
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss * batch_size / len(iterator), epoch_acc * batch_size / len(iterator) 

def evaluate_epoch_wl(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        stop = 0
        for label, text, length in iterator:          
            label = label.float().to(device)
            text = text.to(device)
            predictions = model(text, length).squeeze(1)            
            loss = criterion(predictions, label)            
            acc = binary_accuracy(predictions, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()        
    return epoch_loss * batch_size / len(iterator), epoch_acc * batch_size / len(iterator) 

In [74]:
################### Entrenamiento
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch_wl(model2, train_iter,
                                        optimizer2, loss, device)
    valid_loss, valid_acc = evaluate_epoch_wl(model2, val_iter, loss, device)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.621 | Train Acc: 64.56%
	 Val. Loss: 0.468 |  Val. Acc: 79.20%
Epoch: 02 | Epoch Time: 0m 42s
	Train Loss: 0.437 | Train Acc: 80.19%
	 Val. Loss: 0.520 |  Val. Acc: 76.76%
Epoch: 03 | Epoch Time: 0m 39s
	Train Loss: 0.408 | Train Acc: 81.98%
	 Val. Loss: 0.483 |  Val. Acc: 79.94%
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.383 | Train Acc: 83.12%
	 Val. Loss: 0.374 |  Val. Acc: 84.60%
Epoch: 05 | Epoch Time: 0m 39s
	Train Loss: 0.367 | Train Acc: 84.10%
	 Val. Loss: 0.394 |  Val. Acc: 83.20%
Epoch: 06 | Epoch Time: 0m 39s
	Train Loss: 0.356 | Train Acc: 84.57%
	 Val. Loss: 0.316 |  Val. Acc: 86.87%
Epoch: 07 | Epoch Time: 0m 39s
	Train Loss: 0.335 | Train Acc: 85.87%
	 Val. Loss: 0.319 |  Val. Acc: 87.25%
Epoch: 08 | Epoch Time: 0m 39s
	Train Loss: 0.327 | Train Acc: 86.08%
	 Val. Loss: 0.286 |  Val. Acc: 88.54%
Epoch: 09 | Epoch Time: 0m 39s
	Train Loss: 0.311 | Train Acc: 86.87%
	 Val. Loss: 0.289 |  Val. Acc: 88.24%
Epoch: 10 | Epoch T

In [75]:
####################### Evaluación
test_loss, test_acc = evaluate_epoch_wl(model2, test_iter, loss, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.289 | Test Acc: 88.04%
